In [ ]:
from PIL import Image
import numpy as np

# Import FLAIR
from flair import FLAIRModel
import os
import random
import numpy as np
import torch
from tqdm import tqdm
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

data_dir = '../../datasets/fives/all/'

# in_class = ['A fundus photograph of normal.', 'A fundus photograph of diabetic retinopathy.']
in_class = ['normal', 'diabetic retinopathy']
in_dir = ['N/', 'D/']

ood_dir = ['G/','A/']
in_images = []
ood_images = []

per_image = 150
all_gts = []
for i in range(len(in_dir)):
    _in = in_dir[i]
    sub_dir_path = os.path.join(data_dir, _in)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    in_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])
    #in_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
    all_gts.extend([i for x in range(per_image)])

for _ood in ood_dir:
    sub_dir_path = os.path.join(data_dir, _ood)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])
    # ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
# Set model
model = FLAIRModel(from_checkpoint=True)

# Load image and set target categories 
# (if the repo is not cloned, download the image and change the path!)



# Forward FLAIR model to compute similarities
in_probs = []
ind_score = []
all_preds = []

for in_image in tqdm(in_images):
    image = np.array(Image.open(in_image))
    text = ["%s" %x for x in in_class]
    with torch.no_grad():
        probs, logits = model(image, text)
        in_probs.append(probs)
        preds = np.argmax(probs, axis=1)
    all_preds.append(preds)
    ind_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 
print(ind_score)
ood_score = []
for in_image in tqdm(ood_images):
    image = np.array(Image.open(in_image))
    text = ["%s" %x for x in in_class]
    with torch.no_grad():
        probs, logits = model(image, text)
        preds = np.argmax(logits, axis=1)

    ood_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 

from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)
preds_probs = [x[0][1] for x in in_probs]
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(all_gts, preds_probs)

# Print results
print("Confusion Matrix:\n", cm)
print("AUC:", auc)
print("Accuracy:", accuracy)

from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
concat_score = [x for x in ind_score] + [x for x in ood_score]
concat_gt = [0 for x in range(len(ind_score))] + [1 for x in range(len(ood_score))]

fpr, tpr, _ = roc_curve(concat_gt, concat_score)
roc_auc = auc(fpr, tpr)

# 计算AUPR
precision, recall, _ = precision_recall_curve(concat_gt, concat_score)
aupr = average_precision_score(concat_gt, concat_score)

# 计算FPR95
# 找到tpr最接近0.95的索引
idx = np.argmin(np.abs(tpr - 0.95))
fpr95 = fpr[idx]

print(f"AUC: {roc_auc}")
print(f"AUPR: {aupr}")
print(f"FPR95: {fpr95}")

In [ ]:
from PIL import Image
import numpy as np

# Import FLAIR
from flair import FLAIRModel
import os
import random
import numpy as np
import torch
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

data_dir = '../../datasets/fives/all/'

in_class = ['normal', 'diabetic retinopathy']
in_dir = ['N/', 'D/']

ood_dir = ['imagenet/']
in_images = []
ood_images = []

per_image = 150
all_gts = []
for i in range(len(in_dir)):
    _in = in_dir[i]
    sub_dir_path = os.path.join(data_dir, _in)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    in_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])
    #in_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
    all_gts.extend([i for x in range(per_image)])

for _ood in ood_dir:
    sub_dir_path = os.path.join(data_dir, _ood)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:300]])
# Set model
model = FLAIRModel(from_checkpoint=True)

# Load image and set target categories 
# (if the repo is not cloned, download the image and change the path!)



# Forward FLAIR model to compute similarities
in_probs = []
ind_score = []
all_preds = []

for in_image in tqdm(in_images):
    image = np.array(Image.open(in_image))
    text = ["%s" %x for x in in_class]
    with torch.no_grad():
        probs, logits = model(image, text)
        in_probs.append(probs)
        preds = np.argmax(probs, axis=1)
    all_preds.append(preds)
    ind_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 
print(ind_score)
ood_score = []
for in_image in tqdm(ood_images):
    image = np.array(Image.open(in_image).convert('RGB'))
    text = ["%s" %x for x in in_class]
    with torch.no_grad():
        probs, logits = model(image, text)
        preds = np.argmax(logits, axis=1)

    ood_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 

from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)
preds_probs = [x[0][1] for x in in_probs]
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(all_gts, preds_probs)

# Print results
print("Confusion Matrix:\n", cm)
print("AUC:", auc)
print("Accuracy:", accuracy)

from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
concat_score = [x for x in ind_score] + [x for x in ood_score]
concat_gt = [0 for x in range(len(ind_score))] + [1 for x in range(len(ood_score))]

fpr, tpr, _ = roc_curve(concat_gt, concat_score)
roc_auc = auc(fpr, tpr)

# 计算AUPR
precision, recall, _ = precision_recall_curve(concat_gt, concat_score)
aupr = average_precision_score(concat_gt, concat_score)

# 计算FPR95
# 找到tpr最接近0.95的索引
idx = np.argmin(np.abs(tpr - 0.95))
fpr95 = fpr[idx]

print(f"AUC: {roc_auc}")
print(f"AUPR: {aupr}")
print(f"FPR95: {fpr95}")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 2))
plt.hist(ind_score, bins=50, alpha=0.5, label='In-distribution')
plt.hist(ood_score, bins=50, alpha=0.5, label='Out-of-distribution')
plt.xticks([])
plt.yticks([])
plt.savefig('imagenet.png', pad_inches=0)


In [ ]:
from PIL import Image
import numpy as np

# Import FLAIR
from flair import FLAIRModel
import os
import random
import numpy as np
import torch
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

data_dir = '../../datasets/fives/all/'

in_class = ['normal', 'diabetic retinopathy']
in_dir = ['N/', 'D/']

ood_dir = ['uwf/']
in_images = []
ood_images = []

per_image = 150
all_gts = []
for i in range(len(in_dir)):
    _in = in_dir[i]
    sub_dir_path = os.path.join(data_dir, _in)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    in_images.extend([os.path.join(sub_dir_path, x) for x in  _images[:per_image]])
    #in_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
    all_gts.extend([i for x in range(per_image)])

for _ood in ood_dir:
    sub_dir_path = os.path.join(data_dir, _ood)
    _images = os.listdir(sub_dir_path)
    random.shuffle(_images)
    ood_images.extend([os.path.join(sub_dir_path, x) for x in  _images])
# Set model
model = FLAIRModel(from_checkpoint=True)

# Load image and set target categories 
# (if the repo is not cloned, download the image and change the path!)



# Forward FLAIR model to compute similarities
in_probs = []
ind_score = []
all_preds = []

for in_image in tqdm(in_images):
    image = np.array(Image.open(in_image))
    text = ["%s" %x for x in in_class]
    with torch.no_grad():
        probs, logits = model(image, text)
        in_probs.append(probs)
        preds = np.argmax(probs, axis=1)
    all_preds.append(preds)
    ind_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 
print(ind_score)
ood_score = []
for in_image in tqdm(ood_images):
    image = np.array(Image.open(in_image))
    text = ["%s" %x for x in in_class]
    with torch.no_grad():
        probs, logits = model(image, text)
        preds = np.argmax(logits, axis=1)

    ood_score.append(-torch.logsumexp(torch.from_numpy(logits),dim=1).numpy()) 

from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate confusion matrix
cm = confusion_matrix(all_gts, all_preds)

# Calculate accuracy
accuracy = accuracy_score(all_gts, all_preds)
preds_probs = [x[0][1] for x in in_probs]
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(all_gts, preds_probs)

# Print results
print("Confusion Matrix:\n", cm)
print("AUC:", auc)
print("Accuracy:", accuracy)

from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
concat_score = [x for x in ind_score] + [x for x in ood_score]
concat_gt = [0 for x in range(len(ind_score))] + [1 for x in range(len(ood_score))]

fpr, tpr, _ = roc_curve(concat_gt, concat_score)
roc_auc = auc(fpr, tpr)

# 计算AUPR
precision, recall, _ = precision_recall_curve(concat_gt, concat_score)
aupr = average_precision_score(concat_gt, concat_score)

# 计算FPR95
# 找到tpr最接近0.95的索引
idx = np.argmin(np.abs(tpr - 0.95))
fpr95 = fpr[idx]

print(f"AUC: {roc_auc}")
print(f"AUPR: {aupr}")
print(f"FPR95: {fpr95}")